In [11]:
import sympy.physics.mechanics as mech
from sympy import S,Rational,pi
import sympy as sp

In [14]:
l,t,m,M,g,k,l0,w,r,lam, qref = sp.symbols(r'l t m M g k l0 w \rho \lambda \theta_{ref}')
q1, q2,q3,q4 = mech.dynamicsymbols('q1:5')
q1d, q2d, q3d, q4d = mech.dynamicsymbols('q1:5', 1)

# Create and initialize the reference frame
N = mech.ReferenceFrame('N')
pointN = mech.Point('N*')
pointN.set_vel(N, 0)

# Create the points
point1 = pointN.locatenew('p_1',q1*N.x)
point2 = pointN.locatenew('p_2',q2*N.x)
point3 = point1.locatenew('p_3', l*(sp.sin(q3)*N.x-sp.cos(q3)*N.y))
point4 = point2.locatenew('p_4', l*(sp.sin(q4)*N.x-sp.cos(q4)*N.y))

# Set the points' velocities
point1.set_vel(N, point1.pos_from(pointN).dt(N))
point2.set_vel(N, point2.pos_from(pointN).dt(N))
point3.set_vel(N, point3.pos_from(pointN).dt(N))
point4.set_vel(N, point4.pos_from(pointN).dt(N))

# Create the particles
particle1 = mech.Particle('P_1',point1,M)
particle2 = mech.Particle('P_2',point2,M)
particle3 = mech.Particle('P_3',point3,m)
particle4 = mech.Particle('P_4',point4,m)

# Set the particles' potential energy
particle1.potential_energy = 0
particle2.potential_energy = S.Half*k*((point2.pos_from(pointN)-point1.pos_from(pointN)).magnitude()-l0)**2
particle3.potential_energy = particle3.mass*g*q3
particle4.potential_energy = particle4.mass*g*q4

# Define forces not coming from a potential function
escapement3 = 2*lam*sp.Piecewise((-1, sp.Abs(q3) < qref), (1, sp.Abs(q3) >= qref))
escapement4 = 2*lam*sp.Piecewise((-1, sp.Abs(q4) < qref), (1, sp.Abs(q4) >= qref))


forces=[(point1, -2*r*point1.vel(N)),(point2, -2*r*point2.vel(N)),
        (point3,escapement3*point3.vel(N).normalize()),
        (point4,escapement4*point4.vel(N).normalize())]

# Construct the Lagrangian
L = mech.Lagrangian(N, particle1,particle2,particle3,particle4)

# Create the LagrangesMethod object
LM = mech.LagrangesMethod(L, [q1, q2,q3,q4], hol_coneqs=None, forcelist=forces, frame=N)

# Form Lagranges Equations
ELeqns = LM.form_lagranges_equations()
sp.simplify(ELeqns)

# # Holonomic Constraint Equations
# f_c = Matrix([q1**2 + q2**2 - L**2])

Matrix([
[Piecewise(((2*\lambda*(l*cos(q3(t))*Derivative(q3(t), t) + Derivative(q1(t), t))*(q1(t) - q2(t)) - k*(l0 - sqrt((q1(t) - q2(t))**2))*sqrt(l**2*Derivative(q3(t), t)**2 + 2*l*cos(q3(t))*Derivative(q1(t), t)*Derivative(q3(t), t) + Derivative(q1(t), t)**2)*sqrt((q1(t) - q2(t))**2) + (q1(t) - q2(t))*(M*Derivative(q1(t), (t, 2)) + 2*\rho*Derivative(q1(t), t) + m*(-l*sin(q3(t))*Derivative(q3(t), t)**2 + l*cos(q3(t))*Derivative(q3(t), (t, 2)) + Derivative(q1(t), (t, 2))))*sqrt(l**2*Derivative(q3(t), t)**2 + 2*l*cos(q3(t))*Derivative(q1(t), t)*Derivative(q3(t), t) + Derivative(q1(t), t)**2))/((q1(t) - q2(t))*sqrt(l**2*Derivative(q3(t), t)**2 + 2*l*cos(q3(t))*Derivative(q1(t), t)*Derivative(q3(t), t) + Derivative(q1(t), t)**2)), \theta_{ref} > Abs(q3(t))), ((-2*\lambda*(l*cos(q3(t))*Derivative(q3(t), t) + Derivative(q1(t), t))*(q1(t) - q2(t)) - k*(l0 - sqrt((q1(t) - q2(t))**2))*sqrt(l**2*Derivative(q3(t), t)**2 + 2*l*cos(q3(t))*Derivative(q1(t), t)*Derivative(q3(t), t) + Derivative(q1(

In [18]:
from numpy import array, linspace, sin, cos
from pydy.system import System
import numpy as np

sys = System(LM,constants={
    m:1.0,M:10.0,l:1.0,
    l0:1.0,k: 10.0,g:9.81,
    w:1.0,r:0.2,lam:0.1,
    qref:0.1},
             initial_conditions={
                 q1:0,q2:w,q3:np.pi/4,
                 q4:np.pi/5,q1d:0,q2d:0,
                 q3d:0,q4d:0},
             times=linspace(0.0, 10.0, 1000))
y = sys.integrate()

AttributeError: 'LagrangesMethod' object has no attribute '_uaux'